In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

# Load the dataset
df = pd.read_csv("../../data.csv")

# Split data into X and y
X = df.drop("outcome", axis=1)
y = df["outcome"]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data
scalar = StandardScaler().fit(X_train)
X_train = scalar.transform(X_train)
X_test = scalar.transform(X_test)

# Initialize models
models = {
    "Random Forest": RandomForestClassifier(),
    "XGB": XGBClassifier()
}

# Define hyperparameter grids
rf_grid = {
    "n_estimators": [10, 50, 100],
    "max_depth": [None, 3, 5, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

xgb_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.2, 0.3],
    'subsample': [0.7, 0.9, 1.0],
    'min_child_weight': [1, 2, 3]
}

# Perform Randomized Search CV
rs_rf = RandomizedSearchCV(RandomForestClassifier(), param_distributions=rf_grid, cv=5, n_iter=10, verbose=True)
rs_rf.fit(X_train, y_train)

rs_xgb = RandomizedSearchCV(XGBClassifier(), param_distributions=xgb_grid, cv=5, n_iter=10, verbose=True)
rs_xgb.fit(X_train, y_train)

# Save the best XGBoost model
best_xgb_model_filename = '../../model/best_xgb_model.json'
rs_xgb.best_estimator_.save_model(best_xgb_model_filename)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
